In [ ]:
import pandas as pd
import numpy as np
from ast import literal_eval
import base64
import array
import matplotlib.pyplot as plt
import datetime
import os
from sklearn.model_selection import StratifiedGroupKFold
from torch.utils.data import Dataset,DataLoader
from torchsampler import ImbalancedDatasetSampler
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import models
from torchvision import utils
import matplotlib.pyplot as plt
%matplotlib inline
import time
import copy
import random
from sklearn import metrics
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_curve,auc
from torchsummary import summary
from tableone import TableOne
import copy
from sklearn.model_selection import train_test_split
from lifelines.utils import concordance_index
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold



device='cuda'

# Analysis

In [ ]:
# load inference outputs only
df_test=pd.read_csv('20240701_test_inference.csv')
df_YSH=pd.read_csv('20240701_YSH_inference.csv')
df_AUMC=pd.read_csv('20240701_AUMC_inference.csv')
df_val=pd.read_csv('20240701_val_inference.csv')

label0=[]
for i in range(len(df_test)):
    if df_test['CACS'][i]>0:
        label0.append(1)
    else:
        label0.append(0)
df_test['label0']=label0

label0=[]
for i in range(len(df_val)):
    if df_val['CACS'][i]>0:
        label0.append(1)
    else:
        label0.append(0)
df_val['label0']=label0

label0=[]
for i in range(len(df_YSH)):
    if df_YSH['CACS'][i]>0:
        label0.append(1)
    else:
        label0.append(0)
df_YSH['label0']=label0

label0=[]
for i in range(len(df_AUMC)):
    if df_AUMC['score_float'][i]>0:
        label0.append(1)
    else:
        label0.append(0)
df_AUMC['label0']=label0

In [ ]:
AIpredictiontotallist=[]
for i in range(25):
    AIpredictiontotallist.append('AI_prediction_'+str(i))
    
df_test['AI_average']=df_test[AIpredictiontotallist].mean(axis=1)
df_YSH['AI_average']=df_YSH[AIpredictiontotallist].mean(axis=1)
df_AUMC['AI_average']=df_AUMC[AIpredictiontotallist].mean(axis=1)
df_val['AI_average']=df_val[AIpredictiontotallist].mean(axis=1)

print('==============')
print('df val')
fpr_val,tpr_val,thresholds=metrics.roc_curve(df_val['label400'],df_val['AI_average'],pos_label=1)
AUROC_val=metrics.auc(fpr_val,tpr_val)
print(AUROC_val)

precision_val, recall_val, _ = metrics.precision_recall_curve(df_val['label400'], df_val['AI_average'], pos_label=1)
AUPRC_val = metrics.auc(recall_val, precision_val)
print("AUPRC_val:", AUPRC_val)

youden_j = tpr_val - fpr_val
max_j_index = np.argmax(youden_j)
optimal_threshold = thresholds[max_j_index]


print('========================')
print('df test')
print('=====')
print('label400')
fpr_test,tpr_test,threshold=metrics.roc_curve(df_test['label400'],df_test['AI_average'],pos_label=1)
AUROC_test=metrics.auc(fpr_test,tpr_test)
print(AUROC_test)

precision_test, recall_test, _ = metrics.precision_recall_curve(df_test['label400'], df_test['AI_average'], pos_label=1)
AUPRC_test = metrics.auc(recall_test, precision_test)
print("AUPRC_test:", AUPRC_test)

df_test['predicted'] = (df_test['AI_average'] >= optimal_threshold).astype(int)
tn, fp, fn, tp = metrics.confusion_matrix(df_test['label400'], df_test['predicted']).ravel()
accuracy = metrics.accuracy_score(df_test['label400'], df_test['predicted'])
sensitivity = tp / (tp + fn)  # also known as recall
specificity = tn / (tn + fp)
ppv = tp / (tp + fp)  # positive predictive value (precision)
npv = tn / (tn + fn)  # negative predictive value
f1_score = metrics.f1_score(df_test['label400'], df_test['predicted'])
print(f"Accuracy: {accuracy}")
print(f"Sensitivity: {sensitivity}")
print(f"Specificity: {specificity}")
print(f"PPV (Precision): {ppv}")
print(f"NPV: {npv}")
print(f"F1 Score: {f1_score}")
print('=====')
print('label0')
fpr_test,tpr_test,threshold=metrics.roc_curve(df_test['label0'],df_test['AI_average'],pos_label=1)
AUROC_test=metrics.auc(fpr_test,tpr_test)
print(AUROC_test)

precision_test, recall_test, _ = metrics.precision_recall_curve(df_test['label0'], df_test['AI_average'], pos_label=1)
AUPRC_test = metrics.auc(recall_test, precision_test)
print("AUPRC_test:", AUPRC_test)

df_test['predicted'] = (df_test['AI_average'] >= optimal_threshold).astype(int)
tn, fp, fn, tp = metrics.confusion_matrix(df_test['label0'], df_test['predicted']).ravel()
accuracy = metrics.accuracy_score(df_test['label0'], df_test['predicted'])
sensitivity = tp / (tp + fn)  # also known as recall
specificity = tn / (tn + fp)
ppv = tp / (tp + fp)  # positive predictive value (precision)
npv = tn / (tn + fn)  # negative predictive value
f1_score = metrics.f1_score(df_test['label0'], df_test['predicted'])
print(f"Accuracy: {accuracy}")
print(f"Sensitivity: {sensitivity}")
print(f"Specificity: {specificity}")
print(f"PPV (Precision): {ppv}")
print(f"NPV: {npv}")
print(f"F1 Score: {f1_score}")



print('========================')
print('df YSH')
print('=====')
print('label400')
fpr_YSH,tpr_YSH,threshold=metrics.roc_curve(df_YSH['label400'],df_YSH['AI_average'],pos_label=1)
AUROC_YSH=metrics.auc(fpr_YSH,tpr_YSH)
print(AUROC_YSH)

precision_YSH, recall_YSH, _ = metrics.precision_recall_curve(df_YSH['label400'], df_YSH['AI_average'], pos_label=1)
AUPRC_YSH = metrics.auc(recall_YSH, precision_YSH)
print("AUPRC_YSH:", AUPRC_YSH)

df_YSH['predicted'] = (df_YSH['AI_average'] >= optimal_threshold).astype(int)
tn, fp, fn, tp = metrics.confusion_matrix(df_YSH['label400'], df_YSH['predicted']).ravel()
accuracy = metrics.accuracy_score(df_YSH['label400'], df_YSH['predicted'])
sensitivity = tp / (tp + fn)  # also known as recall
specificity = tn / (tn + fp)
ppv = tp / (tp + fp)  # positive predictive value (precision)
npv = tn / (tn + fn)  # negative predictive value
f1_score = metrics.f1_score(df_YSH['label400'], df_YSH['predicted'])
print(f"Accuracy: {accuracy}")
print(f"Sensitivity: {sensitivity}")
print(f"Specificity: {specificity}")
print(f"PPV (Precision): {ppv}")
print(f"NPV: {npv}")
print(f"F1 Score: {f1_score}")


print('=====')
print('label0')
fpr_YSH,tpr_YSH,threshold=metrics.roc_curve(df_YSH['label0'],df_YSH['AI_average'],pos_label=1)
AUROC_YSH=metrics.auc(fpr_YSH,tpr_YSH)
print(AUROC_YSH)

precision_YSH, recall_YSH, _ = metrics.precision_recall_curve(df_YSH['label0'], df_YSH['AI_average'], pos_label=1)
AUPRC_YSH = metrics.auc(recall_YSH, precision_YSH)
print("AUPRC_YSH:", AUPRC_YSH)

df_YSH['predicted'] = (df_YSH['AI_average'] >= optimal_threshold).astype(int)
tn, fp, fn, tp = metrics.confusion_matrix(df_YSH['label0'], df_YSH['predicted']).ravel()
accuracy = metrics.accuracy_score(df_YSH['label0'], df_YSH['predicted'])
sensitivity = tp / (tp + fn)  # also known as recall
specificity = tn / (tn + fp)
ppv = tp / (tp + fp)  # positive predictive value (precision)
npv = tn / (tn + fn)  # negative predictive value
f1_score = metrics.f1_score(df_YSH['label0'], df_YSH['predicted'])
print(f"Accuracy: {accuracy}")
print(f"Sensitivity: {sensitivity}")
print(f"Specificity: {specificity}")
print(f"PPV (Precision): {ppv}")
print(f"NPV: {npv}")
print(f"F1 Score: {f1_score}")

print('========================')
print('df AUMC')
print('=====')
print('label400')
fpr_AUMC,tpr_AUMC,threshold=metrics.roc_curve(df_AUMC['label400'],df_AUMC['AI_average'],pos_label=1)
AUROC_AUMC=metrics.auc(fpr_AUMC,tpr_AUMC)
print(AUROC_AUMC)

precision_AUMC, recall_AUMC, _ = metrics.precision_recall_curve(df_AUMC['label400'], df_AUMC['AI_average'], pos_label=1)
AUPRC_AUMC = metrics.auc(recall_AUMC, precision_AUMC)
print("AUPRC_AUMC:", AUPRC_AUMC)

df_AUMC['predicted'] = (df_AUMC['AI_average'] >= optimal_threshold).astype(int)
tn, fp, fn, tp = metrics.confusion_matrix(df_AUMC['label400'], df_AUMC['predicted']).ravel()
accuracy = metrics.accuracy_score(df_AUMC['label400'], df_AUMC['predicted'])
sensitivity = tp / (tp + fn)  # also known as recall
specificity = tn / (tn + fp)
ppv = tp / (tp + fp)  # positive predictive value (precision)
npv = tn / (tn + fn)  # negative predictive value
f1_score = metrics.f1_score(df_AUMC['label400'], df_AUMC['predicted'])
print(f"Accuracy: {accuracy}")
print(f"Sensitivity: {sensitivity}")
print(f"Specificity: {specificity}")
print(f"PPV (Precision): {ppv}")
print(f"NPV: {npv}")
print(f"F1 Score: {f1_score}")

print('=====')
print('label0')
fpr_AUMC,tpr_AUMC,threshold=metrics.roc_curve(df_AUMC['label0'],df_AUMC['AI_average'],pos_label=1)
AUROC_AUMC=metrics.auc(fpr_AUMC,tpr_AUMC)
print(AUROC_AUMC)

precision_AUMC, recall_AUMC, _ = metrics.precision_recall_curve(df_AUMC['label0'], df_AUMC['AI_average'], pos_label=1)
AUPRC_AUMC = metrics.auc(recall_AUMC, precision_AUMC)
print("AUPRC_AUMC:", AUPRC_AUMC)

df_AUMC['predicted'] = (df_AUMC['AI_average'] >= optimal_threshold).astype(int)
tn, fp, fn, tp = metrics.confusion_matrix(df_AUMC['label0'], df_AUMC['predicted']).ravel()
accuracy = metrics.accuracy_score(df_AUMC['label0'], df_AUMC['predicted'])
sensitivity = tp / (tp + fn)  # also known as recall
specificity = tn / (tn + fp)
ppv = tp / (tp + fp)  # positive predictive value (precision)
npv = tn / (tn + fn)  # negative predictive value
f1_score = metrics.f1_score(df_AUMC['label0'], df_AUMC['predicted'])
print(f"Accuracy: {accuracy}")
print(f"Sensitivity: {sensitivity}")
print(f"Specificity: {specificity}")
print(f"PPV (Precision): {ppv}")
print(f"NPV: {npv}")
print(f"F1 Score: {f1_score}")

In [ ]:
AUROCs=[]
for i in range(25):
    fpr_test,tpr_test,threshold=metrics.roc_curve(df_test['label0'],df_test['AI_prediction_'+str(i)],pos_label=1)
    AUROC_test=metrics.auc(fpr_test,tpr_test)
    AUROCs.append(AUROC_test)

In [ ]:
print(np.mean(AUROCs))
print(np.std(AUROCs))

In [ ]:
AUROCs=[]
for i in range(25):
    fpr_test,tpr_test,threshold=metrics.roc_curve(df_test['label400'],df_test['AI_prediction_'+str(i)],pos_label=1)
    AUROC_test=metrics.auc(fpr_test,tpr_test)
    AUROCs.append(AUROC_test)

In [ ]:
print(np.mean(AUROCs))
print(np.std(AUROCs))

## ROC curves

In [ ]:
fpr_test,tpr_test,threshold=metrics.roc_curve(df_test['label400'],df_test['AI_average'],pos_label=1)
AUROC_test=metrics.auc(fpr_test,tpr_test)

fpr_YSH,tpr_YSH,threshold=metrics.roc_curve(df_YSH['label400'],df_YSH['AI_average'],pos_label=1)
AUROC_YSH=metrics.auc(fpr_YSH,tpr_YSH)
print(AUROC_YSH)

fpr_AUMC,tpr_AUMC,threshold=metrics.roc_curve(df_AUMC['label400'],df_AUMC['AI_average'],pos_label=1)
AUROC_AUMC=metrics.auc(fpr_AUMC,tpr_AUMC)
print(AUROC_AUMC)

plt.figure(figsize=(10,10))
plt.xlim([-0.00, 1.00])
plt.ylim([-0.00, 1.00])
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='grey',alpha=.8)


plt.plot(fpr_test, tpr_test, color='blue',label='Test AUROC %0.3f' % (AUROC_test) ,lw=1, alpha=1)
plt.plot(fpr_YSH, tpr_YSH, color='red',label=r'External validation (YSH) AUROC %0.3f' % (AUROC_YSH) ,lw=1, alpha=1)
plt.plot(fpr_AUMC, tpr_AUMC, color='green',label=r'External validation (AUMC) AUROC %0.3f' % (AUROC_AUMC) ,lw=1, alpha=1)
plt.xlabel('1-Specificity', fontsize=17)
plt.ylabel('Sensitivity', fontsize=17)
plt.title('ROC curves', fontsize=19)
#plt.legend(loc="lower right", fontsize=29)
plt.xticks(np.arange(0.2, 1.2, step=0.2))
plt.yticks(np.arange(0, 1.2, step=0.2))
plt.xticks(fontsize =15)
plt.yticks(fontsize =15)
leg=plt.legend(loc="lower right", fontsize=17)
for legobj in leg.legendHandles:
    legobj.set_linewidth(2.0)
#plt.show()

plt.savefig('figures/20240702_ROC400.png',transparent=True)

In [ ]:
fpr_test,tpr_test,threshold=metrics.roc_curve(df_test['label0'],df_test['AI_average'],pos_label=1)
AUROC_test=metrics.auc(fpr_test,tpr_test)

fpr_YSH,tpr_YSH,threshold=metrics.roc_curve(df_YSH['label0'],df_YSH['AI_average'],pos_label=1)
AUROC_YSH=metrics.auc(fpr_YSH,tpr_YSH)
print(AUROC_YSH)

fpr_AUMC,tpr_AUMC,threshold=metrics.roc_curve(df_AUMC['label0'],df_AUMC['AI_average'],pos_label=1)
AUROC_AUMC=metrics.auc(fpr_AUMC,tpr_AUMC)
print(AUROC_AUMC)

plt.figure(figsize=(10,10))
plt.xlim([-0.00, 1.00])
plt.ylim([-0.00, 1.00])
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='grey',alpha=.8)


plt.plot(fpr_test, tpr_test, color='blue',label='Test AUROC %0.3f' % (AUROC_test) ,lw=1, alpha=1)
plt.plot(fpr_YSH, tpr_YSH, color='red',label=r'External validation (YSH) AUROC %0.3f' % (AUROC_YSH) ,lw=1, alpha=1)
plt.plot(fpr_AUMC, tpr_AUMC, color='green',label=r'External validation (AUMC) AUROC %0.3f' % (AUROC_AUMC) ,lw=1, alpha=1)
plt.xlabel('1-Specificity', fontsize=17)
plt.ylabel('Sensitivity', fontsize=17)
plt.title('ROC curves', fontsize=19)
#plt.legend(loc="lower right", fontsize=29)
plt.xticks(np.arange(0.2, 1.2, step=0.2))
plt.yticks(np.arange(0, 1.2, step=0.2))
plt.xticks(fontsize =15)
plt.yticks(fontsize =15)
leg=plt.legend(loc="lower right", fontsize=17)
for legobj in leg.legendHandles:
    legobj.set_linewidth(2.0)
#plt.show()

plt.savefig('figures/20240702_ROC_CAC0.png',transparent=True)

## PR curves

In [ ]:
precision_test, recall_test, _ = metrics.precision_recall_curve(df_test['label400'], df_test['AI_average'], pos_label=1)
AUPRC_test = metrics.auc(recall_test, precision_test)
print("AUPRC_test:", AUPRC_test)

precision_YSH, recall_YSH, _ = metrics.precision_recall_curve(df_YSH['label400'], df_YSH['AI_average'], pos_label=1)
AUPRC_YSH = metrics.auc(recall_YSH, precision_YSH)
print("AUPRC_YSH:", AUPRC_YSH)

precision_AUMC, recall_AUMC, _ = metrics.precision_recall_curve(df_AUMC['label400'], df_AUMC['AI_average'], pos_label=1)
AUPRC_AUMC = metrics.auc(recall_AUMC, precision_AUMC)
print("AUPRC_AUMC:", AUPRC_AUMC)

plt.figure(figsize=(10,10))
plt.xlim([-0.00, 1.00])
plt.ylim([-0.00, 1.00])

plt.plot(recall_test, precision_test, color='blue', label='Test AUPRC %0.3f' % (AUPRC_test), lw=1, alpha=1)
plt.plot(recall_YSH, precision_YSH, color='red', label=r'External validation (YSH) AUPRC %0.3f' % (AUPRC_YSH), lw=1, alpha=1)
plt.plot(recall_AUMC, precision_AUMC, color='green', label=r'External validation (AUMC) AUPRC %0.3f' % (AUPRC_AUMC), lw=1, alpha=1)

plt.xlabel('Recall', fontsize=17)
plt.ylabel('Precision', fontsize=17)
plt.title('PR curves', fontsize=19)
plt.xticks(np.arange(0.2, 1.2, step=0.2))
plt.yticks(np.arange(0, 1.2, step=0.2))
plt.xticks(fontsize =15)
plt.yticks(fontsize =15)
leg=plt.legend(loc="upper right", fontsize=17)
for legobj in leg.legendHandles:
    legobj.set_linewidth(2.0)

plt.savefig('figures/20240702_PR_CAC400.png',transparent=True)

In [ ]:
precision_test, recall_test, _ = metrics.precision_recall_curve(df_test['label0'], df_test['AI_average'], pos_label=1)
AUPRC_test = metrics.auc(recall_test, precision_test)
print("AUPRC_test:", AUPRC_test)

precision_YSH, recall_YSH, _ = metrics.precision_recall_curve(df_YSH['label0'], df_YSH['AI_average'], pos_label=1)
AUPRC_YSH = metrics.auc(recall_YSH, precision_YSH)
print("AUPRC_YSH:", AUPRC_YSH)

precision_AUMC, recall_AUMC, _ = metrics.precision_recall_curve(df_AUMC['label0'], df_AUMC['AI_average'], pos_label=1)
AUPRC_AUMC = metrics.auc(recall_AUMC, precision_AUMC)
print("AUPRC_AUMC:", AUPRC_AUMC)

plt.figure(figsize=(10,10))
plt.xlim([-0.00, 1.00])
plt.ylim([-0.00, 1.00])

plt.plot(recall_test, precision_test, color='blue', label='Test AUPRC %0.3f' % (AUPRC_test), lw=1, alpha=1)
plt.plot(recall_YSH, precision_YSH, color='red', label=r'External validation (YSH) AUPRC %0.3f' % (AUPRC_YSH), lw=1, alpha=1)
plt.plot(recall_AUMC, precision_AUMC, color='green', label=r'External validation (AUMC) AUPRC %0.3f' % (AUPRC_AUMC), lw=1, alpha=1)

plt.xlabel('Recall', fontsize=17)
plt.ylabel('Precision', fontsize=17)
plt.title('PR curves', fontsize=19)
plt.xticks(np.arange(0.2, 1.2, step=0.2))
plt.yticks(np.arange(0, 1.2, step=0.2))
plt.xticks(fontsize =15)
plt.yticks(fontsize =15)
leg=plt.legend(loc="upper right", fontsize=17)
for legobj in leg.legendHandles:
    legobj.set_linewidth(2.0)

plt.savefig('figures/20240702_PR_CAC0.png',transparent=True)

## Boxplot of AI average by CACS group

In [ ]:
bins = [-0.1, 0.1, 100, 400, 1000, float('inf')]  # -0.1 to include 0 as a separate bin
labels = ['0', '0-100', '100-400', '400-1000', '1000+']

# Create a new column for the CACS group
df_test['CACS_group'] = pd.cut(df_test['CACS'], bins=bins, labels=labels, right=False)

# Create the boxplot with customized outlier dot size, color, and transparency
plt.figure(figsize=(20, 12))
boxplot = df_test.boxplot(column='AI_average', by='CACS_group', grid=False, 
                     flierprops=dict(marker='o', color='grey', alpha=0.6, markersize=0.6))

# Customize the plot
plt.title(' ', fontsize=13)
plt.suptitle('') 
plt.xlabel('CAC score group', fontsize=11)
plt.ylabel('AI-enabled ECG model prediction score', fontsize=11)
plt.xticks(fontsize=11)
plt.yticks(fontsize=11)

# Show the plot
plt.savefig('figures/20240702_Boxplot_of_AI_average_by_CACS_group.png',transparent=True)

# PCE calculation

In [ ]:
def AHA_calculator(Race, Sex, age, total_chol, hdl_chol, sbp, on_hypertension_meds, smoker, diabetes):
    
    # Selecting coefficients based on race and sex
    if Race == 1 and Sex == 0:
        C_Age = -29.799
        C_Sq_Age = 4.884
        C_Total_Chol = 13.54
        C_Age_Total_Chol = -3.114
        C_HDL_Chol = -13.578
        C_Age_HDL_Chol = 3.149
        C_On_HypertensionMeds = 2.019
        C_Age_On_HypertensionMeds = 0
        C_Off_HypertensionMeds = 1.957
        C_Age_Off_HypertensionMeds = 0
        C_Smoker = 7.574
        C_Age_Smoker = -1.665
        C_Diabetes = 0.661
        S10 = 0.9665
        Mean_Terms = -29.18
    
    elif Race == 1 and Sex == 1:
        C_Age = 12.344
        C_Sq_Age = 0
        C_Total_Chol = 11.853
        C_Age_Total_Chol = -2.664
        C_HDL_Chol = -7.99
        C_Age_HDL_Chol = 1.769
        C_On_HypertensionMeds = 1.797
        C_Age_On_HypertensionMeds = 0
        C_Off_HypertensionMeds = 1.764
        C_Age_Off_HypertensionMeds = 0
        C_Smoker = 7.837
        C_Age_Smoker = -1.795
        C_Diabetes = 0.658
        S10 = 0.9144
        Mean_Terms = 61.18
    
    elif Race == 0 and Sex == 0:
        C_Age = 17.114
        C_Sq_Age = 0
        C_Total_Chol = 0.94
        C_Age_Total_Chol = 0
        C_HDL_Chol = -18.92
        C_Age_HDL_Chol = 4.475
        C_On_HypertensionMeds = 29.291
        C_Age_On_HypertensionMeds = -6.432
        C_Off_HypertensionMeds = 27.82
        C_Age_Off_HypertensionMeds = -6.087
        C_Smoker = 0.691
        C_Age_Smoker = 0
        C_Diabetes = 0.874
        S10 = 0.9533
        Mean_Terms = 86.61
    
    elif Race == 0 and Sex == 1:
        C_Age = 2.469
        C_Sq_Age = 0
        C_Total_Chol = 0.302
        C_Age_Total_Chol = 0
        C_HDL_Chol = -0.307
        C_Age_HDL_Chol = 0
        C_On_HypertensionMeds = 1.916
        C_Age_On_HypertensionMeds = 0
        C_Off_HypertensionMeds = 1.809
        C_Age_Off_HypertensionMeds = 0
        C_Smoker = 0.549
        C_Age_Smoker = 0
        C_Diabetes = 0.645
        S10 = 0.8954
        Mean_Terms = 19.54
    else:
        print('wrong')
        
    #risk calculation(%)
    terms = (C_Age * math.log(age)) + \
        (C_Sq_Age * math.pow(math.log(age), 2)) + \
        (C_Total_Chol * math.log(total_chol)) + \
        (C_Age_Total_Chol * math.log(age) * math.log(total_chol)) + \
        (C_HDL_Chol * math.log(hdl_chol)) + \
        (C_Age_HDL_Chol * math.log(age) * math.log(hdl_chol)) + \
        (on_hypertension_meds * C_On_HypertensionMeds * math.log(sbp)) + \
        (on_hypertension_meds * C_Age_On_HypertensionMeds * math.log(age) * math.log(sbp)) + \
        ((1 - on_hypertension_meds) * C_Off_HypertensionMeds * math.log(sbp)) + \
        ((1 - on_hypertension_meds) * C_Age_Off_HypertensionMeds * math.log(age) * math.log(sbp)) + \
        (C_Smoker * smoker) + \
        (C_Age_Smoker * math.log(age) * smoker) + \
        (C_Diabetes * diabetes)

    risk = 100 * (1 - math.pow(S10,(math.exp(terms - Mean_Terms))))
    risk=round(risk, 1)
    return risk


sexint=[]
for i in range(len(df_test)):
    if df_test['sex'][i]=='M':
        sexint.append(1)
    else:
        sexint.append(0)
df_test['sexint']=sexint

import math
PCE=[]
for i in range(len(df_test)):
    score=AHA_calculator(1,df_test['sexint'][i],df_test['year_difference'][i],df_test['Chol'][i],df_test['HDL'][i],df_test['SBP'][i],df_test['HTN'][i],df_test['smoking'][i],df_test['DM'][i]   )
    PCE.append(score)
    if np.isnan(score)==True:
        print(df.iloc[i])
df_test['PCE']=PCE

In [ ]:
df_test_PCE=df_test.dropna(subset=['PCE'])
df_test_PCE.reset_index(inplace=True,drop=True)

In [ ]:
PCEcat=[]
for i in range(len(df_test_PCE)):
    if df_test_PCE['PCE'][i]<7.5:
        PCEcat.append(1)
    elif df_test_PCE['PCE'][i]<20:
        PCEcat.append(2)
    else:
        PCEcat.append(3)
df_test_PCE['PCEcat']=PCEcat

In [ ]:
df_test_PCE_low=df_test_PCE[df_test_PCE['PCE']<7.5]
df_test_PCE_mod=df_test_PCE[df_test_PCE['PCE']>=7.5]
df_test_PCE_mod=df_test_PCE_mod[df_test_PCE_mod['PCE']<20]
df_test_PCE_hig=df_test_PCE[df_test_PCE['PCE']>=20]

# performances

In [ ]:
thres1= #enter threshold for low vs mod/high risk
thres2= #enter threshold for low/mod vs high risk

columnslist=[]
columnslist.append('CAClevel')
columnslist.append('institution')
columnslist.append('threshold')
columnslist.append('AUROC')
columnslist.append('AUPRC')
columnslist.append('accuracy')
columnslist.append('sensitivity')
columnslist.append('specificity')
columnslist.append('PPV')
columnslist.append('NPV')
columnslist.append('F1score')
performance_df=pd.DataFrame(columns=columnslist)


for labelname in ['label0','label400']:
    for institutiontemp in ['val','SH','YSH','AUMC','df_test_PCE_low','df_test_PCE_mod','df_test_PCE_hig']:
        if institutiontemp=='SH':
            df=df_test.copy()
        elif institutiontemp=='YSH':
            df=df_YSH.copy()        
        elif institutiontemp=='val':
            df=df_val.copy() 
        elif institutiontemp=='df_test_PCE_low':
            df=df_test_PCE_low.copy() 
        elif institutiontemp=='df_test_PCE_mod':
            df=df_test_PCE_mod.copy() 
        elif institutiontemp=='df_test_PCE_hig':
            df=df_test_PCE_hig.copy() 
        else:
            df=df_AUMC.copy()
        
        for thresholdtemp in [thres1,thres2]:
            templist=[]
            templist.append(labelname)
            templist.append(institutiontemp)
            templist.append(thresholdtemp)
            
            fpr,tpr,threshold=metrics.roc_curve(df[labelname],df['AI_average'],pos_label=1)
            AUROC=metrics.auc(fpr,tpr)
            precision, recall, _ = metrics.precision_recall_curve(df[labelname], df['AI_average'], pos_label=1)
            AUPRC = metrics.auc(recall, precision)

            df['predicted'] = (df['AI_average'] >= thresholdtemp).astype(int)
            tn, fp, fn, tp = metrics.confusion_matrix(df[labelname], df['predicted']).ravel()
            accuracy = metrics.accuracy_score(df[labelname], df['predicted'])
            sensitivity = tp / (tp + fn)  # also known as recall
            specificity = tn / (tn + fp)
            ppv = tp / (tp + fp)  # positive predictive value (precision)
            npv = tn / (tn + fn)  # negative predictive value
            f1_score = metrics.f1_score(df[labelname], df['predicted'])

            templist.append(AUROC)
            templist.append(AUPRC)
            templist.append(accuracy)
            templist.append(sensitivity)
            templist.append(specificity)
            templist.append(ppv)
            templist.append(npv)
            templist.append(f1_score)
            performance_df.loc[len(performance_df)]=templist

## Survival analysis

In [ ]:
dateuntil=365*9
cutlength=90

df=pd.read_csv('20240701_survivaldata.csv',encoding='cp949')

df['checkup_date'] = pd.to_datetime(df['checkup_date'])
df['date2'] = pd.to_datetime(df['date2'])
df.loc[df['date2'] > pd.Timestamp('2022-12-31'), 'date2'] = pd.NaT

df['status'] = df['date2'].notnull().astype(int)
df=df[~(df['checkup_date']  >= df['date2'])]
df['day_difference'] = df.apply(lambda row: (row['date2'] - row['checkup_date']).days if row['status'] == 1 else (pd.Timestamp('2022-12-31') - row['checkup_date']).days, axis=1)

df = df[(df['day_difference'] >= cutlength) | df['day_difference'].isna()]
df = df.reset_index(drop=True)
df['day_difference'] = df['day_difference'] - cutlength


for i in range(len(df)):
    if df['status'][i]==1 and df['day_difference'][i]>=dateuntil:
        df['status'][i]=0
        df['day_difference'][i]=dateuntil
    if df['status'][i]==0 and df['day_difference'][i]>=dateuntil:
        df['day_difference'][i]=dateuntil

In [ ]:
# Assuming df is already loaded and the columns are prepared as per the previous code
# Calculate person-time in years
df['person_years'] = df['day_difference'] / 365.25

# Calculate total number of events
total_events = df['status'].sum()

# Calculate total person-time
total_person_time = df['person_years'].sum()

# Calculate incidence rate per 1000 person-years
incidence_rate = (total_events / total_person_time) * 1000

print(f"Incidence rate per 1000 person-years: {incidence_rate}")

In [ ]:
df2=df[df['PCEcat']==1]
print(len(df2))
df2['person_years'] = df2['day_difference'] / 365.25
total_events = df2['status'].sum()
total_person_time = df2['person_years'].sum()
incidence_rate = (total_events / total_person_time) * 10
print(f"Incidence rate per person-10years: {incidence_rate}")

df2=df[df['PCEcat']==2]
print(len(df2))
df2['person_years'] = df2['day_difference'] / 365.25
total_events = df2['status'].sum()
total_person_time = df2['person_years'].sum()
incidence_rate = (total_events / total_person_time) * 10
print(f"Incidence rate per person-10years: {incidence_rate}")

df2=df[df['PCEcat']==3]
print(len(df2))
df2['person_years'] = df2['day_difference'] / 365.25
total_events = df2['status'].sum()
total_person_time = df2['person_years'].sum()
incidence_rate = (total_events / total_person_time) * 10
print(f"Incidence rate per person-10years: {incidence_rate}")

In [ ]:
print(np.percentile(df['person_years'],25))
print(np.percentile(df['person_years'],50))
print(np.percentile(df['person_years'],75))

In [ ]:
AIcat=[]
for i in range(len(df)):
    if df['AI_prediction_average'][i]>=thres2:
        AIcat.append(3)
    elif df['AI_prediction_average'][i]>=thres1:
        AIcat.append(2)
    else:
        AIcat.append(1)
df['AIcat']=AIcat

AIcatnum=1
df2=df[df['AIcat']==AIcatnum]
df2['person_years'] = df2['day_difference'] / 365.25
total_events = df2['status'].sum()
total_person_time = df2['person_years'].sum()
incidence_rate = (total_events / total_person_time) * 10
print(f"total Incidence rate per person-10years: {incidence_rate}")
print(total_events)
print(total_person_time)

AIcatnum=2
df2=df[df['AIcat']==AIcatnum]
df2['person_years'] = df2['day_difference'] / 365.25
total_events = df2['status'].sum()
total_person_time = df2['person_years'].sum()
incidence_rate = (total_events / total_person_time) * 10
print(f"total Incidence rate per person-10years: {incidence_rate}")
print(total_events)
print(total_person_time)

AIcatnum=3
df2=df[df['AIcat']==AIcatnum]
df2['person_years'] = df2['day_difference'] / 365.25
total_events = df2['status'].sum()
total_person_time = df2['person_years'].sum()
incidence_rate = (total_events / total_person_time) * 10
print(f"total Incidence rate per person-10years: {incidence_rate}")
print(total_events)
print(total_person_time)

In [ ]:
def kmcurve(df):
    TopTertile = df[df['AI_prediction_average'] >thres2]
    MiddleTertile= df[df['AI_prediction_average'] <=thres2]
    MiddleTertile= MiddleTertile[MiddleTertile['AI_prediction_average'] >thres1]
    BottomTertile = df[df['AI_prediction_average'] <=thres1]

    kmf = KaplanMeierFitter()
    plt.figure(figsize=(10, 8))

    kmf.fit(TopTertile['day_difference'], TopTertile['status'])
    ci_lower = 1 - kmf.confidence_interval_['KM_estimate_upper_0.95']
    ci_upper = 1 - kmf.confidence_interval_['KM_estimate_lower_0.95']
    plt.fill_between(kmf.survival_function_.index, ci_lower, ci_upper, color='red',alpha=0.3)
    plt.step(kmf.survival_function_.index, 1 - kmf.survival_function_, color='red',label='High risk (AI)')

    kmf.fit(MiddleTertile['day_difference'], MiddleTertile['status'])
    ci_lower = 1 - kmf.confidence_interval_['KM_estimate_upper_0.95']
    ci_upper = 1 - kmf.confidence_interval_['KM_estimate_lower_0.95']
    plt.fill_between(kmf.survival_function_.index, ci_lower, ci_upper, color='green',alpha=0.3)
    plt.step(kmf.survival_function_.index, 1 - kmf.survival_function_, color='green',label='Moderate risk (AI)')


    kmf.fit(BottomTertile['day_difference'], BottomTertile['status'])
    ci_lower = 1 - kmf.confidence_interval_['KM_estimate_upper_0.95']
    ci_upper = 1 - kmf.confidence_interval_['KM_estimate_lower_0.95']
    plt.fill_between(kmf.survival_function_.index, ci_lower, ci_upper, color='blue',alpha=0.3)
    plt.step(kmf.survival_function_.index, 1 - kmf.survival_function_, color='blue',label='Low risk (AI)')

    plt.xlim(0, dateuntil)  # Set x-axis range
    plt.ylim([0, 0.048])  # Adjust the limits as per your data

    xlabel_size = 22  # Adjust as needed
    ylabel_size = 22  # Adjust as needed
    ticks_size = 20   # Adjust as needed
    title_size = 23   # Adjust as needed
    legend_size = 18  # Adjust as needed

    plt.xlabel('Time (days)', fontsize=xlabel_size)
    plt.ylabel('Cumulative Incidence', fontsize=ylabel_size)
    plt.xticks(np.arange(0, dateuntil, step=500), fontsize=ticks_size)  # Adjust step as per your data
    plt.yticks(np.arange(0, 0.048, step=0.005), fontsize=ticks_size)

    leg=plt.legend(loc='upper left', fontsize=legend_size)
    for legobj in leg.legendHandles:
        legobj.set_linewidth(2.5)


    plt.savefig('figures/20240709_survival_tertile.png',transparent=True)

    # Perform the multivariate logrank test
    T = pd.concat([BottomTertile['day_difference'], MiddleTertile['day_difference'], TopTertile['day_difference']])
    E = pd.concat([BottomTertile['status'], MiddleTertile['status'], TopTertile['status']])
    groups = ['Bottom'] * len(BottomTertile) + ['Middle'] * len(MiddleTertile) + ['Top'] * len(TopTertile)

    result = multivariate_logrank_test(T, groups, E)
    print(result.print_summary())
    print(result.p_value)


    T1=BottomTertile['day_difference']
    T2=MiddleTertile['day_difference']
    E1=BottomTertile['status']
    E2=MiddleTertile['status']
    logrankresults=logrank_test(T1,T2,event_observed_A=E1,event_observed_B=E2)
    print(logrankresults.print_summary())
    print(logrankresults.p_value)

    T1=MiddleTertile['day_difference']
    T2=TopTertile['day_difference']
    E1=MiddleTertile['status']
    E2=TopTertile['status']
    logrankresults=logrank_test(T1,T2,event_observed_A=E1,event_observed_B=E2)
    print(logrankresults.print_summary())
    print(logrankresults.p_value)

    df['person_years'] = df['day_difference'] / 365.25
    total_events = df['status'].sum()
    total_person_time = df['person_years'].sum()
    incidence_rate = (total_events / total_person_time) * 10
    print(f"total Incidence rate per person-10years: {incidence_rate}")

    TopTertile['person_years'] = TopTertile['day_difference'] / 365.25
    total_events = TopTertile['status'].sum()
    total_person_time = TopTertile['person_years'].sum()
    incidence_rate = (total_events / total_person_time) * 10
    print(f"Top Incidence rate per person-10years: {incidence_rate}")
    print(total_events)
    print(total_person_time)

    MiddleTertile['person_years'] = MiddleTertile['day_difference'] / 365.25
    total_events = MiddleTertile['status'].sum()
    total_person_time = MiddleTertile['person_years'].sum()
    incidence_rate = (total_events / total_person_time) * 10
    print(f"Middle Incidence rate per person-10years: {incidence_rate}")
    print(total_events)
    print(total_person_time)

    BottomTertile['person_years'] = BottomTertile['day_difference'] / 365.25
    total_events = BottomTertile['status'].sum()
    total_person_time = BottomTertile['person_years'].sum()
    incidence_rate = (total_events / total_person_time) * 10
    print(f"Bottom Incidence rate per person-10years: {incidence_rate}")
    print(total_events)
    print(total_person_time)

In [ ]:
PCEcatnum=1
df2=df[df['PCEcat']==PCEcatnum]
kmcurve(df2)

In [ ]:
PCEcatnum=2
df2=df[df['PCEcat']==PCEcatnum]
kmcurve(df2)

# NRI, c-index

In [ ]:
df_final=df.copy()
for statusname in ['status']:
    print('============',statusname,'============')
    NRI=0
    for i in [0,1]:
        if i==0:
            print('neg')
        else:
            print('pos')
        dftemp=df_final[df_final[statusname]==i]
        print('total',len(dftemp))

        AI_low_PCE_low=dftemp[dftemp['PCE_ECG_class']==1]
        AI_low_PCE_low=AI_low_PCE_low[AI_low_PCE_low['PCEclass']==1]
        AI_low_PCE_mod=dftemp[dftemp['PCE_ECG_class']==1]
        AI_low_PCE_mod=AI_low_PCE_mod[AI_low_PCE_mod['PCEclass']==2]
        AI_hig_PCE_low=dftemp[dftemp['PCE_ECG_class']==2]
        AI_hig_PCE_low=AI_hig_PCE_low[AI_hig_PCE_low['PCEclass']==1]
        AI_hig_PCE_mod=dftemp[dftemp['PCE_ECG_class']==2]
        AI_hig_PCE_mod=AI_hig_PCE_mod[AI_hig_PCE_mod['PCEclass']==2]
        
        totalsum=len(AI_low_PCE_low)+len(AI_low_PCE_mod)+len(AI_hig_PCE_low)+len(AI_hig_PCE_mod)
        
        print('PCEECG_low_PCE_low',len(AI_low_PCE_low))
        print('PCEECG_low_PCE_mod',len(AI_low_PCE_mod),len(AI_low_PCE_mod)/totalsum)
        print('PCEECG_hig_PCE_low',len(AI_hig_PCE_low),len(AI_hig_PCE_low)/totalsum)
        print('PCEECG_hig_PCE_mod',len(AI_hig_PCE_mod))
        
        print('totalsum',totalsum)
        if i==1:
            print('positive NRI',(len(AI_hig_PCE_low)-len(AI_low_PCE_mod))/totalsum)
            NRI+=(len(AI_hig_PCE_low)-len(AI_low_PCE_mod))/totalsum
        else:
            print('negative NRI',(len(AI_low_PCE_mod)-len(AI_hig_PCE_low))/totalsum)
            NRI+=(len(AI_low_PCE_mod)-len(AI_hig_PCE_low))/totalsum

    print(NRI)

In [ ]:
NRIdf_columns=[]
NRIdf_columns.append('neg_PCElow_ECGmod')
NRIdf_columns.append('neg_PCEmod_ECGlow')
NRIdf_columns.append('neg_NRI')
NRIdf_columns.append('pos_PCElow_ECGmod')
NRIdf_columns.append('pos_PCEmod_ECGlow')
NRIdf_columns.append('pos_NRI')
NRIdf_columns.append('total_NRI')

NRIdf=pd.DataFrame(columns=NRIdf_columns)

for jjj in range(2000):
    df_final_temp=df_final.sample(frac=1, replace=True)
    
    templist=[]
    NRI=0
    for i in [0,1]:
        dftemp=df_final_temp[df_final_temp[statusname]==i]
        #print('total',len(dftemp))

        AI_low_PCE_low=dftemp[dftemp['PCE_ECG_class']==1]
        AI_low_PCE_low=AI_low_PCE_low[AI_low_PCE_low['PCEclass']==1]
        AI_low_PCE_mod=dftemp[dftemp['PCE_ECG_class']==1]
        AI_low_PCE_mod=AI_low_PCE_mod[AI_low_PCE_mod['PCEclass']==2]
        AI_hig_PCE_low=dftemp[dftemp['PCE_ECG_class']==2]
        AI_hig_PCE_low=AI_hig_PCE_low[AI_hig_PCE_low['PCEclass']==1]
        AI_hig_PCE_mod=dftemp[dftemp['PCE_ECG_class']==2]
        AI_hig_PCE_mod=AI_hig_PCE_mod[AI_hig_PCE_mod['PCEclass']==2]

        totalsum=len(AI_low_PCE_low)+len(AI_low_PCE_mod)+len(AI_hig_PCE_low)+len(AI_hig_PCE_mod)


        #print('totalsum',totalsum)
        if i==1:
            NRI+=(len(AI_hig_PCE_low)-len(AI_low_PCE_mod))/totalsum
        else:
            NRI+=(len(AI_low_PCE_mod)-len(AI_hig_PCE_low))/totalsum
        
        templist.append(len(AI_hig_PCE_low)/totalsum)
        templist.append(len(AI_low_PCE_mod)/totalsum)
        if i==0:
            templist.append((len(AI_low_PCE_mod)-len(AI_hig_PCE_low))/totalsum)
        else:
            templist.append((len(AI_hig_PCE_low)-len(AI_low_PCE_mod))/totalsum)
        
    templist.append(NRI)
    #print(NRI)
    NRIdf.loc[len(NRIdf)]=templist
    if jjj%100==0:
        print(jjj)

In [ ]:
NRIdfcols=list(NRIdf.columns)
for col in NRIdfcols:
    print(col)
    print(np.percentile(NRIdf[col],97.5))
    print(np.percentile(NRIdf[col],2.5))
    print('======================')

In [ ]:
print(concordance_index(df_final['day_difference'], -df_final['PCE'], df_final['status']))
print(concordance_index(df_final['day_difference'], -df_final['PCE_AI'], df_final['status']))

In [ ]:
rows_list=[]
for iiii in range(2000):
    df_final_resample=df_final.sample(frac=1, replace=True)

    cindex_cvd_events_pce=concordance_index(df_final_resample['day_difference'], -df_final_resample['PCE'], df_final_resample['status'])
    cindex_cvd_events_pce_ai=concordance_index(df_final_resample['day_difference'], -df_final_resample['PCE_AI'], df_final_resample['status'])


    row = [
        cindex_cvd_events_pce,
        cindex_cvd_events_pce_ai
    ]
    rows_list.append(row)
    if iiii%100==0:
        print(iiii)
df_C_INDEX = pd.DataFrame(rows_list, columns=['cindex_cvd_events_pce','cindex_cvd_events_pce_ai'])    

In [ ]:
df_C_INDEX['cindex_cvd_events_diff']=df_C_INDEX['cindex_cvd_events_pce_ai']-df_C_INDEX['cindex_cvd_events_pce']

In [ ]:
percentile_value_low = np.percentile(df_C_INDEX['cindex_cvd_events_pce'], 2.5)
percentile_value_high = np.percentile(df_C_INDEX['cindex_cvd_events_pce'], 97.5)
print(percentile_value_low,percentile_value_high)

In [ ]:
percentile_value_low = np.percentile(df_C_INDEX['cindex_cvd_events_pce_ai'], 2.5)
percentile_value_high = np.percentile(df_C_INDEX['cindex_cvd_events_pce_ai'], 97.5)
print(percentile_value_low,percentile_value_high)

In [ ]:
percentile_value_low = np.percentile(df_C_INDEX['cindex_cvd_events_diff'], 2.5)
percentile_value_high = np.percentile(df_C_INDEX['cindex_cvd_events_diff'], 97.5)
print(percentile_value_low,percentile_value_high)

# characteristics table

In [ ]:
AIcat=[]
for i in range(len(df)):
    if df['AI_prediction_average'][i]>=thres2:
        AIcat.append(3)
    elif df['AI_prediction_average'][i]>=thres1:
        AIcat.append(2)
    else:
        AIcat.append(1)
df['AIcat']=AIcat

In [ ]:
statcols=[]
statcols.append('year_difference')
statcols.append('sexint')
statcols.append('DM')
statcols.append('HTN')
statcols.append('smoking')
statcols.append('SBP')
statcols.append('Chol')
statcols.append('HDL')
statcols.append('AI_prediction_average')
statcols.append('PCE')
statcols.append('status')
statcols.append('AIcat')


catcols=[]
catcols.append('sexint')
catcols.append('DM')
catcols.append('HTN')
catcols.append('smoking')
catcols.append('status')
catcols.append('AIcat')

In [ ]:
df_stats=df[statcols]

In [ ]:
def table1(catcolumn,catcolumns,dfdfdf2):
    nonnormallist=[]
    def _normality(self, x):
        #print(x.name)

        if len(x.values[~np.isnan(x.values)]) >= 20:
            p = stats.shapiro(x.values).pvalue
        else:
            p = None
        # dropna=False argument in pivot_table does not function as expected
        # return -1 instead of None
        if pd.isnull(p):
            return -1
        if p<=0.05:
            nonnormallist.append(x.name)
        return p

    TableOne._normality=_normality

    def my_custom_test(group1, group2):
        """
        Hypothesis test for test_self_defined_statistical_tests
        """
        my_custom_test.__name__ = "mannwhitneyu"
        _, pval= scipy.stats.mannwhitneyu(group1, group2)
        return pval

    nonnormallist=[]
    def _normality(self, x):
        #print(x.name)

        if len(x.values[~np.isnan(x.values)]) >= 20:
            p = stats.shapiro(x.values).pvalue
        else:
            p = None
        # dropna=False argument in pivot_table does not function as expected
        # return -1 instead of None
        if pd.isnull(p):
            return -1
        if p<=0.05:
            nonnormallist.append(x.name)
        return p

    TableOne._normality=_normality

    table1=TableOne(dfdfdf2,categorical=catcolumns,groupby=[catcolumn],normal_test=True,pval=True,htest_name=True,decimals=3)
    nonnormallist=list(set(nonnormallist))
    nonnormallist


    table1=TableOne(dfdfdf2,categorical=catcolumns,groupby=[catcolumn],normal_test=True,pval=True,htest_name=True,nonnormal=nonnormallist,decimals=3)
    try:
        os.mkdir(newtablename)
    except:
        pass
    try:
        os.mkdir(newtablename+'/table1')
    except:
        pass
    catcolumn=catcolumn.replace(' ','_')
    catcolumn=catcolumn.replace('/','_')
    table1.to_html('figures/20241001_SH_table1_'+catcolumn+'.html')

In [ ]:
table1('AIcat',catcols,df_stats)